# Imitation Learning Dataset Generation Notebook

This notebook demonstrates how to generate additional datasets through imitation learning using Isaac Lab Mimic. The workflow consists of several key steps:

1. **Configure basic parameters**: 
   - Task name
   - Number of parallel environments
   - Number of demonstrations to generate
2. **Initialize the environment**: Instantiates the IsaacLab environment with configured parameters
2. **Interactive Parameter Updates**: Allows customization of randomizable parameters
3. **Data Generation**: Generates new demonstrations based on existing annotated datasets


## Initial Configuration

This cell sets up the basic configuration for data generation:

1. **How to Modify**:
   - Change `task` to your desired environment
   - Adjust `num_envs` based on your GPU capability
   - Set `generation_num_trials` to how many demos you want

2. **Tips**:
   - Start with 10 trials for testing, increase for training

In [ ]:
from notebook_widgets import create_task_input, create_num_trials_input

task = create_task_input()
num_envs = 1
num_trials = create_num_trials_input()

## Environment Setup

Run this cell to initialize the simulation environment. This sets up the necessary components for data generation.

In [ ]:
import os
import nest_asyncio
nest_asyncio.apply()

from argparse import ArgumentParser, Namespace
from isaaclab.app import AppLauncher

parser = ArgumentParser()
AppLauncher.add_app_launcher_args(parser)
args_cli = parser.parse_args([])
args_cli.enable_cameras = True
args_cli.kit_args = "--enable omni.videoencoding"

config = {
    "task": task.value,  
    "num_envs": num_envs,                                       
    "generation_num_trials": num_trials.value,                         
    "input_file": "datasets/annotated_dataset.hdf5",     
    "output_file": "datasets/generated_dataset.hdf5", 
    "pause_subtask": False,
    "enable": "omni.kit.renderer.capture",
}

# Update the default configuration
args_dict = vars(args_cli)
args_dict.update(config)
args_cli = Namespace(**args_dict)

# Now launch the simulator with the final configuration
app_launcher = AppLauncher(args_cli)
simulation_app = app_launcher.app

import asyncio
import gymnasium as gym
import numpy as np
import random
import torch

import isaaclab_mimic.envs  # noqa: F401
from isaaclab_mimic.datagen.generation import env_loop, setup_env_config, setup_async_generation
from isaaclab_mimic.datagen.utils import get_env_name_from_dataset, setup_output_paths, interactive_update_randomizable_params, reset_env
from isaaclab.managers import ObservationTermCfg as ObsTerm
from notebook_utils import ISAACLAB_OUTPUT_DIR

import isaaclab_tasks  # noqa: F401
num_envs = args_cli.num_envs

# Setup output paths and get env name
output_dir, output_file_name = setup_output_paths(args_cli.output_file)
env_name = args_cli.task or get_env_name_from_dataset(args_cli.input_file)

# Configure environment
env_cfg, success_term = setup_env_config(
    env_name=env_name,
    output_dir=output_dir,
    output_file_name=output_file_name,
    num_envs=num_envs,
    device=args_cli.device,
    generation_num_trials=args_cli.generation_num_trials,
)
# Set observation output directory
for obs in vars(env_cfg.observations.rgb_camera).values():
    if not isinstance(obs, ObsTerm):
        continue
    obs.params["image_path"] = os.path.join(ISAACLAB_OUTPUT_DIR, obs.params["image_path"])
env_cfg.observations


# create environment
env = gym.make(env_name, cfg=env_cfg)

# set seed for generation
random.seed(env.unwrapped.cfg.datagen_config.seed)
np.random.seed(env.unwrapped.cfg.datagen_config.seed)
torch.manual_seed(env.unwrapped.cfg.datagen_config.seed)

# reset before starting
reset_env(env, 100)

## Interactive Parameter Updates

This section provides interactive sliders and controls to adjust various environment parameters in real-time:

1. **What You'll See**:
   - Sliders for numerical values
   - Range inputs for min/max settings
   - Current value displays
   - Parameter names and allowed ranges

2. **How to Use**:
   - Move the sliders to adjust values
   - Watch the environment update in real-time

3. **Tips**:
   - Start with small adjustments to understand their effects

Note: These adjustments will affect how new demonstrations are generated, so take time to experiment with different settings to achieve desired behavior.

In [ ]:
for event_term in env.unwrapped.event_manager._mode_term_cfgs["reset"]:
    if hasattr(event_term, "randomizable_params") and event_term.randomizable_params is not None:
        print(f"Updating parameters for event: {event_term.func.__name__}")
        interactive_update_randomizable_params(event_term, event_term.randomizable_params, env=env)

## Data Generation

Run this cell to start generating demonstrations using the parameters you've configured. The process will:
- Generate the specified number of demonstrations
- Save successful demonstrations to your output file
- Show progress as demonstrations are generated

In [ ]:
# Setup and run async data generation
async_components = setup_async_generation(
    env=env,
    num_envs=args_cli.num_envs,
    input_file=args_cli.input_file,
    success_term=success_term,
    pause_subtask=args_cli.pause_subtask
)

try:
    asyncio.ensure_future(asyncio.gather(*async_components['tasks']))
    env_loop(env, async_components['action_queue'], 
            async_components['info_pool'], async_components['event_loop'])
except asyncio.CancelledError:
    print("Tasks were cancelled.")


# Cosmos
## Video Generation
The normals are used to apply shading to the semantic segmentation which produces an input that works very well for the Cosmos model that will be used.

In [ ]:
from notebook_widgets import create_camera_input, create_start_frame_input
from notebook_utils import ISAACLAB_OUTPUT_DIR, COSMOS_OUTPUT_DIR

VIDEO_LENGTH = 120   # Suggested length is between 120 and 200
START_FRAME = 0

camera_selection = create_camera_input(ISAACLAB_OUTPUT_DIR)
start_frame = create_start_frame_input(ISAACLAB_OUTPUT_DIR)

In [ ]:
import os
from IPython.display import Video
from notebook_utils import encode_video, ISAACLAB_OUTPUT_DIR

video_filepath = os.path.join(ISAACLAB_OUTPUT_DIR, "shaded_segmentation.mp4")
encode_video(ISAACLAB_OUTPUT_DIR, start_frame.value, VIDEO_LENGTH, camera_selection.value, video_filepath)

display(Video(video_filepath))

## Cosmos Generation
### Get API Keys
#### NVIDIA NGC Catalog key
The NVIDIA NGC API Key is used in this blueprint in order to pull the default router models from NGC as well as the NVIDIA Triton Inference Server docker image which is used to run those models. Refer to [Generating NGC API Keys](https://docs.nvidia.com/ngc/gpu-cloud/ngc-user-guide/index.html#generating-api-key) in the NVIDIA NGC User Guide for more information.

#### NVIDIA API Catalog key
The LLM router is responsible for routing requests between foundational LLM models. In this example, we will use NVIDIA NIMs. In order to access these LLMs we will use a NVIDIA API Catalog key, which is separate from the NVIDIA NGC API key used above. You can alternatively use other LLM models or even on-premise models. To do so, you would update the router configuration file which will be discussed later.

1. Navigate to **[NVIDIA API Catalog](https://build.nvidia.com/explore/discover)**.
2. Select a model, such as cosmos-1.0-diffusion-7b.
3. Select an **Input** option. The following example is of a model that offers a Docker option. Not all of the models offer this option, but all include a “Get API Key” link
<img src="https://docscontent.nvidia.com/dims4/default/d6307a8/2147483647/strip/true/crop/1920x919+0+0/resize/2880x1378!/format/webp/quality/90/?url=https%3A%2F%2Fk3-prod-nvidia-docs.s3.us-west-2.amazonaws.com%2Fbrightspot%2Fsphinx%2F00000192-bfa6-da2c-a1f2-ffbf41aa0000%2Fnim%2Flarge-language-models%2Flatest%2F_images%2Fbuild_docker_tab.png">
4. Click **Get API Key**.
<img src="https://docscontent.nvidia.com/dims4/default/c6e2096/2147483647/strip/true/crop/1920x919+0+0/resize/2880x1378!/format/webp/quality/90/?url=https%3A%2F%2Fk3-prod-nvidia-docs.s3.us-west-2.amazonaws.com%2Fbrightspot%2Fsphinx%2F00000192-bfa6-da2c-a1f2-ffbf41aa0000%2Fnim%2Flarge-language-models%2Flatest%2F_images%2Fbuild_get_api_key.png">
5. Select **\"Generate Key\"**
<img src="https://docscontent.nvidia.com/dims4/default/e7c4057/2147483647/strip/true/crop/1920x919+0+0/resize/2880x1378!/format/webp/quality/90/?url=https%3A%2F%2Fk3-prod-nvidia-docs.s3.us-west-2.amazonaws.com%2Fbrightspot%2Fsphinx%2F00000192-bfa6-da2c-a1f2-ffbf41aa0000%2Fnim%2Flarge-language-models%2Flatest%2F_images%2Fbuild_generate_key.png">
6. **Copy your key** and store it in a secure place. Do not share it.
<img src="https://docscontent.nvidia.com/dims4/default/4b0710a/2147483647/strip/true/crop/1920x919+0+0/resize/2880x1378!/format/webp/quality/90/?url=https%3A%2F%2Fk3-prod-nvidia-docs.s3.us-west-2.amazonaws.com%2Fbrightspot%2Fsphinx%2F00000192-bfa6-da2c-a1f2-ffbf41aa0000%2Fnim%2Flarge-language-models%2Flatest%2F_images%2Fbuild_copy_key.png">

In [ ]:
import ipywidgets as widgets
api_key_widget = widgets.Text(value="", placeholder="Enter API Key Here", description="API Key:", style={'description_width': 'initial'})
display(api_key_widget)

### Using the Cosmos Model

The Cosmos model is parametrized with several available parameters which alter the output in various ways:
- prompt: Text prompt for the video generation.
- seed: Seed for the random number generator.
- control_weight: A float value between 0 and 1 that controls the strength of the controlnet transformation.
- sigma_max: A float value representing the maximum sigma. Lower values result in less change from the original input while a larger values allows for more change but may diverge more from the input scene.
- use_canny_edge: A boolean indicating whether to use the canny edges from the input video.
- blur_strength: A string representing the blur strength.

In [ ]:
from notebook_widgets import create_variable_dropdowns, create_cosmos_params
from notebook_utils import ISAACLAB_OUTPUT_DIR, COSMOS_OUTPUT_DIR

prompt_manager = create_variable_dropdowns("stacking_prompt.toml")
cosmos_params = create_cosmos_params(ISAACLAB_OUTPUT_DIR)

## Generate with Cosmos
---
**NOTE:** Generation generally takes between 1 to 3 minutes

---

### Tips
- To increase prompt adherence, try increasing the `Sigma Max` value and/or `Blur Strength`
- To reduce divergence from the input scene, try increasing the `Control Weight` and/or decreasing `Sigma Max`

In [ ]:
import os
from nvcf_async import api_call_async
from notebook_utils import ISAACLAB_OUTPUT_DIR
from notebook_widgets import create_download_link
from IPython.display import Video

params = {k: w.value for k, w in cosmos_params.items()}
video_filepath = os.path.join(ISAACLAB_OUTPUT_DIR, params.pop("input_video"))
params["prompt"] = f"\"{prompt_manager.prompt}\""
params["input_video"] = "image_0"
params["blur_strength"] = f"\"{params['blur_strength'].lower().replace(' ', '_')}\""

if not api_key_widget.value:
    raise ValueError("Enter a valid API Key to proceed.")

if not os.path.exists(video_filepath):
    raise ValueError(f"Video file not found at {video_filepath}")

result = await api_call_async(
    token=api_key_widget.value,
    params=params,
    filepaths=[video_filepath],
)

if result.status_code == 200 and result.output is not None and "video" in result.output:
    # save the output video
    os.makedirs(COSMOS_OUTPUT_DIR, exist_ok=True)
    output_path = f"{COSMOS_OUTPUT_DIR}/cosmos_{params['seed']}.mp4"
    with open(output_path, "wb") as f:
        f.write(result.output["video"])
    print(f"Output video saved to {output_path}")
    display(Video(output_path))
    display(create_download_link(output_path, link_text=f"Download Video: {output_path}"))
else:
    display(f"An unexpected error occurred: {result.response_text}")

In [ ]:
# Close simulation app
simulation_app.close()